In [1]:
import requests
import json
import firebase_admin
from firebase_admin import credentials
from firebase_admin import auth
from firebase_admin import db
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import io
import PIL.Image as Image

In [2]:
#Objeto de credenciais do firebase
cred_obj = credentials.Certificate("C://arquivo com as credenciais.json")
firebase_admin.initialize_app(cred_obj, {
	'databaseURL': 'URL do banco de dados'
})

#Código para pegar o usuário do firebase
email = "email de usuário do firebase"
user = auth.get_user_by_email(email)

#Consulta dos dados no banco de dados
ref = db.reference('/UsersData/'+ str(user.uid)+'/SHT/')
data = ref.order_by_key().get()


In [3]:
#Carregando dados históricos do arquivo JSON
with open('dado_hist.json') as f:
    json_data = f.read()
    
data_hist = json.loads(json_data)


In [4]:
#Conversão dos dados históricos para o formato do banco de dados
data = []

for data_point in data_hist:
    temperatura = data_point['main']['temp']
    umidade = data_point['main']['umidade']
    timestamp = data_point['dt']
    
    data.append({"timestamp": timestamp, "temperatura": temperatura, "umidade": umidade})
    

In [5]:
# Loop para interpolar a cada 5 minutos e suavizar com média móvel
dado_interpolado = []

for i in range(len(data)-1):
    timestamp_inicial = data[i]['timestamp']
    timestamp_final = data[i+1]['timestamp']
    temperatura_inicial = data[i]['temperatura']
    temperatura_final = data[i+1]['temperatura']
    umidade_inicial = data[i]['umidade']
    umidade_final = data[i+1]['umidade']

    # Interpolação linear
    for j in range(1, 12):
        timestamp = timestamp_inicial + j*300
        temperatura = np.interp(timestamp, [timestamp_inicial, timestamp_final], [temperatura_inicial, temperatura_final])
        umidade = np.interp(timestamp, [timestamp_inicial, timestamp_final], [umidade_inicial, umidade_final])
        dado_interpolado.append({"timestamp": timestamp, "temperatura": temperatura, "umidade": umidade})
    

    #Média móvel
    temperatura_suavizada = [dado_interpolado[i+j]['temperatura'] for j in range(1, 11)]
    umidade_suavizada = [dado_interpolado[i+j]['umidade'] for j in range(1, 11)]
    temperatura_media = np.mean(temperatura_suavizada)
    umidade_media = np.mean(umidade_suavizada)

    # Distribuição normal
    temperatura_std = np.std(temperatura_suavizada)
    umidade_std = np.std(umidade_suavizada)
    for j in range(11):
        temperatura = np.random.normal(temperatura_media, temperatura_std)
        umidade = np.random.normal(umidade_media, umidade_std)
        dado_interpolado[i+j]['temperatura'] = round(temperatura, 2)
        dado_interpolado[i+j]['umidade'] = round(umidade, 2)

# Adiciona os dados interpolados e com distribuição normal ao dicionário 'data'
for item in dado_interpolado:
    data.append(item)


data = sorted(data, key=lambda k: k['timestamp'])


In [6]:
#Envia os dados para o banco de dados
for d in data:
    ref.child(str(d['timestamp'])).set({
        "TemperaturaC": str(d['temperatura']),
        "timestamp": str(d['timestamp']),
        "umidade": str(d['umidade'])
    })